Bu ödevinizde verilen ratings.csv dosyasını kullanacaksınız. Bu dosyada kullanıcıların farklı ürünler için verdikleri puanlar vardır. En yüksek puan 10'dur.

Ödevinizde pandas, numpy kütüphanelerini kullanabilirsiniz. Ayrıca grafik oluşturmak için herhangi bir görselleştirme kütüphanesi kullanabilirsiniz. Bunlar dışında herhangi bir kütüphane kullanamazsınız.

Tüm yakınlık hesaplamaları için kosinus benzerliğini kullanınız.

**Sorularda belirtilmeyen bir parametre vs. var ise, o kısım sizin dizayn kararınıza bırakılmıştır demektir. Bu durumlarda aldığınız kararı belirtiniz.**

**Soru 1 [5 puan]** İlgili veri kümesinin “ratings.csv” verisinde satır sayısının mod 10’u sıfır olan satırlar test için gerisi eğitim için kullanılacaktır. Yani, 10. 20., 30., … satırlar test için kullanılacaktır. Eğitim ve test kümelerini belirleyiniz. Satırları saymaya 0'dan başlayınız.

In [ ]:
# cevabınızı buraya yazacaksınız




train = ...
test = ...

**Soru 2 [20 puan]** Item bazlı “Collaborative filtering” yöntemini kullanarak en yakın 5 komşuyu göz önünde alarak test kümesinin ilk beş değeri için yapılan tahmin değerlerini ve gerçek değeri gösteriniz. 

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 3 [20 puan]** Kullanıcı bazlı “Collaborative filtering” yöntemini kullanarak en yakın 5 komşuyu göz önünde alarak test kümesinin ilk beş değeri için yapılan tahmin değerlerini ve gerçek değeri gösteriniz.

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 4 [15 puan]**  Item bazlı “Collaborative filtering” yöntemini kullanarak en yakın komşu sayısı 1, 3, 5, ve 7 olduğunda test kümesi üzerinde RMSE değerlerini hesaplayınız ve sonuçlarınızı bir figür ya da tablo gibi açıkca anlaşılacak bir şekilde gösteriniz.

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 5 [15 puan]**  Kullanıcı bazlı “Collaborative filtering” yöntemini kullanarak en yakın komşu sayısı 1, 3, 5, ve 7 olduğunda test kümesi üzerinde RMSE değerlerini hesaplayınız ve sonuçlarınızı bir figür ya da tablo gibi açıkca anlaşılacak bir şekilde gösteriniz.

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 6 [25 puan]**  Kullanıcıya döndüğünüz ürünlerden ilk 5'inde 9 veya 10 puan olan ürünleri satın alacak. 

a) Bir kullanıcı maksimum 5 ürün alacaksa, test kümesinde maksimum toplam satış kaç olabilir?


b) Önceki sorularda geliştirdiğiniz kullanıcı bazlı “Collaborative filtering” ve Item bazlı “Collaborative filtering” yöntemlerine göre firmanın kazanacağı parayı hesaplayınız. “Collaborative filtering” yöntemlerinde bakılacak en yakın komşu sayısı 5'tir.

c) Firmanın kazandığı parayı da arttıracak şekilde Item bazlı “Collaborative filtering” yöntemini geliştiriniz. Önerdiğiniz yöntem ile ne kadar kazanç olacağını hesaplayınız.

In [ ]:
# cevabınızı buraya yazacaksınız